# New York AirBnB Data
Using the NewYork data - useful facts about the data:

Business Questions (Business Understanding)
<b><li>What are the most common type of  properties in NewYork available for renting?What are the availabilities of the properties throughout the year?What property types have the highest availabilty?</li></b> 

<b><li>What is the average pricing of the properties available for renting?Are there any seasonal spiking in the prices of the properties?</li></b> 

<b><li>What streets have the highest and lowest property prices? 
Where to Invest a Property in NewYork to get maximum returns from Airbnb?</li></b> 

<b><li>How well can we predict the prices of the properties?What price can you get to rent your home,what factors determine the price of your home?</li></b> 

First, let's read in the necessary libraries and datasets.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
listings = pd.read_csv('listings.csv')
calendar = pd.read_csv('calendar.csv')

Little data wrangling and cleaning here, to prepare the data for analysis 

In [ ]:
#droping columns which are not usefull in  analysing the respective questions.
listings = listings.drop(['transit', 'neighborhood_overview', 'listing_url', 'summary', 'space', 'description', 
                          'access', 'interaction', 'house_rules', 'thumbnail_url','medium_url','picture_url', 
                          'xl_picture_url', 'host_url', 'host_name', 'host_about', 'host_thumbnail_url', 'host_verifications',
                          'calendar_updated'], axis = 1)

In [ ]:
#removing columns which has only one value as that column would be not usefull for analysis 
for col in listings:
    if listings[col].value_counts().count() <= 1:
        listings = listings.drop(col, axis = 1)

In [ ]:
#dropping the columns which have 60% of its values missing
for col in listings:
        percentage = listings[col].isna().sum() / listings.shape[0] * 100
        if percentage > 60:
            listings = listings.drop(col, axis = 1)

In [ ]:
#Replacing Nan values with 0
listings.fillna(0, inplace = True)
calendar.fillna(0, inplace = True)

In [ ]:
listings.price.dtype

In [ ]:
calendar.price.dtype

In [ ]:
#converting price into float
listings['price'] = listings['price'].apply(lambda x: float(str(x).replace('$', '').replace(',', '')))
calendar['price'] = calendar['price'].apply(lambda x: float(str(x).replace('$', '').replace(',', '')))

In [ ]:
calendar['date'].head()

In [ ]:
#adding a new column Year-Month to do the monthly analysis of the data
calendar['Year-Month'] = calendar['date'].apply(lambda x:(str(x).replace(x,x[0:7])))
calendar.head()

In [ ]:
calendar.available.dtype

In [ ]:
#converting the column "available" into boolean for performing operations
calendar.available.replace('f', False, inplace = True)
calendar.available.replace('t', True, inplace = True)
calendar.head()

<b>1) What are the most common type of properties in NewYork available for renting?What are the availabilities of the properties throughout the year?What property types have the highest availabilty?</b>